In [ ]:
import pandas as pd
import json
import os
import re
#read xlsx file
df = pd.read_csv('/home/tak/MOJ/resources/data/tagging/drugs/feature_extraction/manual_feature_extraction_gt.csv')
columns = ['שם קובץ התיק','מתחם ענישה שופט']
ranges = df[columns]
for index, row in ranges.iterrows():
    cases = df['שם קובץ התיק']
    current_case = cases[index]
    ranges.at[index, 'שם קובץ התיק'] = current_case
    for column in columns:
        # print(row[column])
        if type(row[column]) == float:
            ranges.at[index, column] = '-'
            continue
            
        if 'שנים' in row[column]:
            row_split = row[column].split()
            # print(row_split)
            if len(row_split) == 4:
                try:
                    row_split[0] = str(int(row_split[0]) * 12)
                except:
                    row_split[0] = str(int(float(row_split[0]) * 12))
                try:
                    row_split[2] = str(int(row_split[2]) * 12)
                except:
                    row_split[2] = str(int(float(row_split[2]) * 12))
                row_split[3] = 'חודשים'
                ginal = ' '.join(map(str, row_split))
        
                ranges.at[index, column] = ginal
            elif len(row_split) == 5:
                #pop the elemne int second index
                row_split.pop(1)
                try:
                    row_split[2] = str(int(row_split[2]) * 12)
                except:
                    row_split[2] = str(int(float(row_split[2]) * 12))
                row_split[3] = 'חודשים'
                ginal = ' '.join(map(str, row_split))
                ranges.at[index, column] = ginal
    path = f'/home/tak/MOJ/results/db/drugs/{current_case}/punishment_range_gpt.json'
    #read json file
    #check if the file exists
    if not os.path.exists(path):
        ranges.at[index, 'extract'] = '-'
        continue
    with open(path, 'r') as f:
        data = json.load(f)
    # if type(data) == dict:
        # if 'lower' in data and 'top' in data:
        #     st = data['lower'] + ' - ' + data['top'] + ' חודשים'
        #     ranges.at[index, 'extract'] = st
        d = data[-1]
        #remove last .
        numbers = re.findall(r'\d+', d)
        result_str  = f'{numbers[0]} - {numbers[1]} חודשים'
        ranges.at[index, 'extract'] = result_str 

    # else:
    #     ranges.at[index, 'extract'] = '-'
    # if ranges['extract'][index] == '' and type(ranges['מתחם ענישה - שופט'][index]) == float:
    #     ranges.at[index, 'same'] = False
    # elif ranges['extract'][index] == ranges['מתחם ענישה - שופט'][index]:
    #     ranges.at[index, 'same'] = True
    # else:
    #     ranges.at[index, 'same'] = False
#orginize the columns
ranges = ranges[['שם קובץ התיק', 'מתחם ענישה שופט','extract']]
# for index, row in ranges.iterrows():
#     if ranges['extract'][index] == '' and type(ranges['מתחם ענישה - שופט'][index]) == float:
#         ranges.at[index, 'same'] = False
#     elif ranges['extract'][index] == ranges['מתחם ענישה - שופט'][index]:
#         ranges.at[index, 'same'] = True
#     else:
#         ranges.at[index, 'same'] = False
# if 'מתחם ענישה - שופט' cilune and 'extract' column are the same, add column with same value else not same value
ranges['same'] = ranges['מתחם ענישה שופט'] == ranges['extract']
if type(ranges['מתחם ענישה שופט']) == float and type(ranges['extract']) == float:
    ranges['same'] = True
#also if both nan it shold be same
#save the new data to the xlsx file
ranges.to_excel('/home/tak/MOJ/src/flows/final_gpt_drugs.xlsx', index=False)

/tmp/ipykernel_1598761/715559630.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranges.at[index, 'שם קובץ התיק'] = current_case
/tmp/ipykernel_1598761/715559630.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranges.at[index, 'extract'] = '-'


In [9]:
df = pd.read_excel('/home/tak/MOJ/src/flows/final.xlsx')
#in column 'same' count how many True and how many False
true_count = df['same'].value_counts().get(True, 0)
false_count = df['same'].value_counts().get(False, 0)
print(f'True: {true_count}')
print(f'False: {false_count}')

#calculate the percentage of True and False
true_percentage = (true_count / len(df)) * 100
false_percentage = (false_count / len(df)) * 100
print(f'True percentage: {true_percentage}%')
print(f'False percentage: {false_percentage}%')


True: 19
False: 52
True percentage: 26.76056338028169%
False percentage: 73.23943661971832%


In [10]:
import os
import pandas as pd
path = '/home/tak/MOJ/results/db/drugs'

for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        split_file = file.split('.')
        if split_file[0] == folder:
            # change name of file to preprocessing.csv
            new_file_path = os.path.join(folder_path, 'preprocessing.csv')
            os.rename(file_path, new_file_path)
            break